In [1]:
import sys
import PySide2
from PySide2 import QtCore, QtGui, QtWidgets
from PySide2.QtWidgets import QApplication, QPushButton ,QWidget , QHBoxLayout ,QVBoxLayout, QFileDialog ,QLineEdit
from PySide2.QtGui import QWindow
import random
import math

points_list=[]
dimensionslist=[]
distancelist=[]

def Import_pic(): 
    fileName = QtWidgets.QFileDialog.getOpenFileName()
    pixmap = QtGui.QPixmap(str(fileName[0]))
    label.setPixmap(pixmap)
    label.setAlignment(QtCore.Qt.AlignLeft)
    dimenArea.show()
    dimenLabel.show()
    enterButton.show()
    
def get_distance():
    x1=int(points_list[0][0])
    y1=int(points_list[0][1])
    x2=int(points_list[1][0])
    y2=int(points_list[1][1])
    dis=math.sqrt((x1-x2)**2+(y1-y2)**2)
#     points_list.pop(0)
#     points_list.pop(1)
#     distancelist.append(dis)
    return dis 
 
    
def convert_to_autocad():
    print("convert done")

def getPos(event):
    x = event.pos().x()
    y = event.pos().y() 
    points_list.append((x,y))
    if(clicklabel.text()=="First point selected"):
        clicklabel.setText("Second point selected ")
    else:
        clicklabel.setText("First point selected")
    clicklabel.show()
        
    
    
def on_enter_button_clicked():
    dimensionslist.append(int(dimenArea.text()))
    dimenArea.clear()
    clicklabel.hide()
    
    
# Create the Qt Application
app = QApplication(sys.argv)
# Create a button
importButton = QPushButton("Import picture")
convertButton = QPushButton("Draw in autocad")


# Connect the button to the function
importButton.clicked.connect(Import_pic)
importButton.setIcon(QtGui.QIcon("./icons8-xlarge-icons-filled-100.png"))
convertButton.clicked.connect(convert_to_autocad)
convertButton.setIcon(QtGui.QIcon("./icons8-edit-512.png"))
importButton.setStyleSheet('QPushButton {background-color: #00CED1; color: black;}')
convertButton.setStyleSheet('QPushButton {background-color:#FFB6C1; color: black;}')





mainwindow = QWidget()

mainwindow = QWidget()
mainwindow.resize(1000, 800)
buttonslayout = QVBoxLayout()
#button.setIcon(QtGui.QIcon('myImage.jpg'))
buttonslayout.addWidget(importButton)
buttonslayout.addWidget(convertButton)

buttonslayout.addStretch(2)

#enter dimensions
dimenArea= QLineEdit()
dimenArea.hide()
#enter dimension label
dimenLabel= QtWidgets.QLabel("Press two points and Enter Dimension")
dimenLabel.hide()
# dimension enter
enterButton=QPushButton("Enter")
enterButton.hide()
enterButton.clicked.connect(on_enter_button_clicked)
# click flag 
clicklabel=QtWidgets.QLabel()
clicklabel.hide()

#pixmap = QtGui.QPixmap(fileName)
label = QtWidgets.QLabel()
label.mousePressEvent = getPos

#textArea and enter layout
lineLayout=QHBoxLayout()
lineLayout.addWidget(dimenArea)
lineLayout.addWidget(enterButton)

#Dimensions and image Layout
DimenLayout = QVBoxLayout()
DimenLayout.addWidget(label)
DimenLayout.addWidget(dimenLabel)
DimenLayout.addLayout(lineLayout)
DimenLayout.addWidget(clicklabel)
DimenLayout.addStretch(2)

#label.setPixmap(pixmap) 

mainlayout = QHBoxLayout()
mainlayout.addLayout(buttonslayout,2)
mainlayout.addLayout(DimenLayout)

mainwindow.setLayout(mainlayout)
mainwindow.show()
# Run the main Qt loop
app.exec_()

0

In [2]:

points_list

[(53, 352), (260, 160)]

In [9]:
dis=get_distance()
dis


282.3349075123372

In [4]:
points_list

[(53, 352), (260, 160)]

In [5]:
# class MyWidget(QtWidgets.QWidget):
#     def __init__(self):
#         super().__init__()

#         #self.hello = ["Hallo Welt", "Hei maailma", "Hola Mundo", "Привет мир"]

#         self.button = QtWidgets.QPushButton("Click me!")
#         fileName="C:/Users/HP/Desktop/ImageGUI/icons8-edit-500.png"
#         pixmap = QtGui.QPixmap(fileName)
    
#         self.text = QtWidgets.QLabel()
#         self.text.setPixmap(pixmap)
        
#         #self.text.setAlignment(QtCore.Qt.AlignCenter)

#         self.layout = QtWidgets.QVBoxLayout()
#         self.layout.addWidget(self.text)
#         self.layout.addWidget(self.button)
#         self.setLayout(self.layout)

#         self.button.clicked.connect(self.magic)


#     def magic(self):
#         #self.text.setText(random.choice(self.hello))
#         fileName = QtWidgets.QFileDialog.getOpenFileName()
#         print(fileName[0])
#         pixmap = QtGui.QPixmap(str(fileName[0]))
#         self.text.setPixmap(pixmap)
#         self.text.show()
#         self.update()
        


In [6]:
# app = QtWidgets.QApplication([])

# widget = MyWidget()
# widget.resize(800, 600)
# widget.show()

# sys.exit(app.exec_())

In [7]:
dimensionslist

[12]